In [1]:
import numpy as np
import pandas as pd
from certa.eval import expl_eval
from certa.explain import explain

from certa.utils import merge_sources
from certa.local_explain import get_original_prediction

In [2]:
model_api = ...
model = model_api.load('models/bert/abt_buy')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [10]:
def predict_fn(x):
    return model.predict(x)

In [4]:
datadir = 'path/to/abt_buy'
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

In [5]:
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], []).dropna()

In [6]:
rand_row = test_df[test_df['label']==1].iloc[4]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]
label = rand_row['label']
label

1

In [7]:
l_tuple

id                                                           631
name           sony 3.1 channel home theater surround system ...
description    sony 3.1 channel home theater surround system ...
price                                                        299
Name: 631, dtype: object

In [8]:
r_tuple

id                                                  864
name                  sony ht-ct100 home theater system
description    , 3.1 speakers progressive scan 250w rms
price                                            279.95
Name: 864, dtype: object

In [11]:
explanation = explain(l_tuple, r_tuple, lsource, rsource, predict_fn, datadir)

/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/

In [35]:
explanation

defaultdict(int,
            {'name': 0.0, 'description': 0.07692307692307693, 'price': 0.0})

In [17]:
prediction = get_original_prediction(l_tuple, r_tuple, predict_fn)
prediction

array([0.993335  , 0.00666496], dtype=float32)

In [36]:
exp = list(explanation)[1]
e_attrs = exp.split('/')
e_score = explanation[exp]

expl_evaluation = expl_eval(np.argmax(prediction), e_attrs, e_score, lsource, l_tuple, model,
                            prediction, rsource,
                            r_tuple, predict_fn)
expl_evaluation




/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


,match,e_score,drop,perturb,flip,impact,attributes,e_size,prediction,row
0,0,0.076923,0.006896,drop,0,0,[description],1,0.993335,631-864
1,0,0.076923,0.000648,sample,0,0,[description],1,0.993335,631-864
